In [ ]:
from agents import User, Agent, Module, Orchestrator

In [ ]:
user = User(
    description="You are a human user interacting with conversational and recommendation agents."
)

In [ ]:
import os
from typing import List

llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-70b-8192",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

user = User(
    description="You are a human user interacting with conversational and recommendation agents."
)

host_viagens = Agent(
    name="host_viagens",
    llm_config=llm_config,
    system_message="""
    You are a friendly travel conversation agent.
    Keep the dialogue natural.
    Do not generate travel recommendations.
""",
    description="External agent that manages travel-related conversations."
)

def get_travel_destinations(preference: str) -> List[str]:
    """
    Returns a list of 3 travel destinations based on an exact preference string match.

    Parameters:
        preference (str): A keyword describing the travel preference
                          (e.g., "cultural", "aventura", "gastronomia")

    Returns:
        List[str]: 3 recommended destinations.
    """
    preference = preference.strip().lower()

    if preference == "cultural":
        return ["Lisbon, Portugal", "Rome, Italy", "Kyoto, Japan"]
    elif preference == "aventura":
        return ["Queenstown, New Zealand", "Patagonia, Chile", "Banff, Canada"]
    elif preference == "praia":
        return ["Maui, Hawaii", "Phuket, Thailand", "Fernando de Noronha, Brazil"]
    elif preference == "história":
        return ["Athens, Greece", "Cairo, Egypt", "Jerusalem, Israel"]
    else:
        return ["Paris, France", "New York City, USA", "Tokyo, Japan"]

perfilador = Agent(
    name="perfilador",
    llm_config=llm_config,
    system_message="""
    Ask short, smart questions to identify the user's travel preferences.
""",
    description="Analyzes user profile and preferences."
)

explainer = Agent(
    name="explainer",
    llm_config=llm_config,
    system_message="""
    You are an explanation agent specialized in justifying travel destination recommendations.

    Your task is to explain *why* the recommended destinations match the user's preference exactly.
    Base your explanation on the input string provided (e.g., "cultural", "aventura", "praia", etc).

    Be specific and informative. Mention how each destination aligns with the user's interest.

    Do not recommend new places or invent new preferences. Focus only on the reasoning behind the current suggestions.
""",
    description="Explains why the recommended destinations fit the user's stated travel preference."
)


recomenda_viagem = Agent(
    name="recomenda_viagem",
    llm_config=llm_config,
    system_message="""
    You recommend 3 travel destinations based on user preferences.
    Always use the 'get_travel_destinations' tool. Do not invent destinations.
    """,
    description="Recommends destinations based on preferences.",
    tools=[get_travel_destinations]
)

recomendacao_modulo = Module(
    admin_name="recomendacao_modulo",
    agents=[user, perfilador, recomenda_viagem, explainer],
    messages=[],
    max_round=3,
    speaker_selection_method="auto",
    allow_repeat_speaker=False,
)

modulo_recomendacao = ConversationalOrchestratorManager(
    name="modulo_recomendacao",
    chitchat=recomendacao_modulo,
    llm_config=llm_config,
    description="This module should only be activated when the USER explicitly requests travel recommendations.",
)

chitchat = ConversationalOrchestrator(
    agents=[user, host_viagens, modulo_recomendacao],
    messages=[],
    max_round=100,
    speaker_selection_method="auto",
    allow_repeat_speaker=False,
)

chitchat_manager = Orchestrator(
    chitchat=chitchat,
    llm_config=llm_config
)

In [ ]:
chat_result = user.to_talk(
    chitchat_manager, message="Oi tudo bem?",
    cache=None
)
#"Estou planejando minhas férias e queria sugestões de lugares com boa comida e cultura."

In [ ]:
chat_result